In [ ]:
"""commented out for test execution"""
# from google.colab import drive
# drive.mount('/content/drive')



#공용함수

In [ ]:
# 수정금지: 타임스탬프용 함수
from datetime import datetime
def printt(*args,**kwargs):
  now = datetime.now()
  now_str = "{:02}:{:02}:{:02}".format(now.hour,now.minute,now.second)
  print(now_str, *args,**kwargs)
  return int(now.hour)*60*60+int(now.minute)*60+int(now.second)

#연관 패키지 설치 (다른작업 금지)

In [ ]:
#TODO: 해당 블럭에 패키지 설치하세요.
!pip install attrdict
!pip install transformers
!pip install seqeval
!pip install fastprogress

# 파일로딩 (다른작업 금지)

In [ ]:
#TODO: 해당 블럭에 필요 파일 로딩 코드 넣으시오.


# 모델 학습

In [ ]:
# 수정금지: 타임스탬프
_model_build_start_time = printt("Model building: Start")
_model_build_start_time

In [ ]:
#TODO: 블럭에 모델 학습 - 빌딩 코드를 넣으세요. (시간측정 구간)





In [ ]:
!sudo apt-get install -y fonts-nanum
!sudo fc-cache -fv
!rm ~/.cache/matplotlib -rf

In [ ]:
# 실행 전 런타임 다시시작
import matplotlib.pyplot as plt
plt.rc('font', family='NanumBarunGothic') 
plt.plot([1, 2, 3, 4])
plt.ylabel('한국어 테스트')
plt.show()

In [ ]:
import os
!git clone https://github.com/Jaehun-Kim22/SCIC_Project.git

In [ ]:
# set config args for classification
from transformers import (
    ElectraConfig,
    ElectraTokenizer,
    ElectraForSequenceClassification,
    XLMRobertaForSequenceClassification,
    XLMRobertaTokenizer,
    XLMRobertaConfig
)
from attrdict import AttrDict
CLASSIFICATION_LIST = (
    ['중립', '상담원', '상담시스템', '혜택', '할부금융상품', 
     '카드상품', '청구입금', '심사/한도', '생활편의서비스', '상담/채널',
     '리스렌탈상품','라이프서비스', '금융상품', '고객정보관리', 
     '삼성카드', '기타']
)
CLASSIFICATION_DICT = {}
for idx, item in enumerate(CLASSIFICATION_LIST):
    CLASSIFICATION_DICT[str(idx)] = item

args = AttrDict(
    {
        'version': 1,
        'data_dir': 'SCIC_Project/assets/data_dup_deleted',
        'train_file': 'train_s_cls.txt',
        'test_file': 'evaluation_cls.txt',
        'task': 'classification',
        'config': ElectraConfig,
        'tokenizer': ElectraTokenizer,
        'model': ElectraForSequenceClassification,
        'evaluate_test_during_training': True, 
        'eval_all_checkpoints': True, 
        'save_optimizer': False, 
        'do_lower_case': False, 
        'do_train': True, 
        'do_eval': True, 
        'max_seq_len': 64, 
        'num_train_epochs': 10, 
        'weight_decay': 0.0, 
        'gradient_accumulation_steps': 1, 
        'adam_epsilon': 1e-08, 
        'warmup_proportion': 0, 
        'max_steps': -1, 
        'max_grad_norm': 1.0, 
        'no_cuda': False, 
        'model_type': 'koelectra-base-v3', 
        'model_name_or_path': 'monologg/koelectra-base-v3-discriminator', 
        'output_dir': 'checkpoints/version_1', 
        'seed': 42, 
        'train_batch_size': 32, 
        'eval_batch_size': 32, 
        'logging_steps': 267, 
        'save_steps': 267, 
        'learning_rate': 5e-05,
        'tb_root': './runs'
     }
)
args.data_dir = os.path.join(args.data_dir, args.task)
args.output_dir = args.task + args.output_dir

In [ ]:
# initialize logging
import logging

logging.basicConfig(
    format= "%(asctime)s - %(message)s",
    # format="%(asctime)s - %(levelname)s - %(name)s -   %(message)s",
    datefmt="%H:%M:%S",
    level=logging.INFO,
)

In [ ]:
import torch
from torch.utils.tensorboard import SummaryWriter

# set seed
torch.cuda.manual_seed_all(args.seed)

In [ ]:
# make Input Example
class InputExample(object):
    """
    A single training/test example for simple sequence classification.
    """

    def __init__(self, review_token, label1, label2):
        self.review_token = review_token
        self.label1 = label1
        self.label2 = label2

    def __repr__(self):
        return str(self.to_json_string())

    def to_dict(self):
        """Serializes this instance to a Python dictionary."""
        output = copy.deepcopy(self.__dict__)
        return output

    def to_json_string(self):
        """Serializes this instance to a JSON string."""
        return json.dumps(self.to_dict(), indent=2, sort_keys=True) + "\n"
        
# make Input Feature
class InputFeatures(object):
    """A single set of features of data."""

    def __init__(self, input_ids, attention_mask, token_type_ids, label1, label2):
        self.input_ids = input_ids
        self.attention_mask = attention_mask
        self.token_type_ids = token_type_ids
        self.label1 = label1
        self.label2 = label2

    def __repr__(self):
        return str(self.to_json_string())

    def to_dict(self):
        """Serializes this instance to a Python dictionary."""
        output = copy.deepcopy(self.__dict__)
        return output

    def to_json_string(self):
        """Serializes this instance to a JSON string."""
        return json.dumps(self.to_dict(), indent=2, sort_keys=True) + "\n"


In [ ]:
if args.task == 'sentiment':
    args.logging_steps, args.save_steps = 63, 63
    args.max_seq_len = 64
    args.train_batch_size = 128
    args.learning_rate = 2e-06

from torch.utils.data import TensorDataset

def load_dataset(args, tokenizer, mode):
    features = get_features(args, tokenizer, mode)

    # Convert to Tensors and build dataset
    all_input_ids = torch.tensor(
        [f.input_ids for f in features], 
        dtype=torch.long)
    all_attention_mask = torch.tensor(
        [f.attention_mask for f in features],
         dtype=torch.long)
    all_token_type_ids = torch.tensor(
        [f.token_type_ids for f in features],
        dtype=torch.long)

    all_labels_1 = torch.tensor([f.label1 for f in features],
                        dtype=torch.float)
    all_labels_2 = torch.tensor([f.label2 for f in features],
                              dtype=torch.long)

    dataset = TensorDataset(all_input_ids,
                            all_attention_mask,
                            all_token_type_ids,
                            all_labels_1,
                            all_labels_2)
    return dataset


def get_features(args, tokenizer, mode):
    # load data and labels using Processor
    processor = Processor(args)
    try:
        examples = processor.get_examples(mode)
    except ValueError:
        print('possible modes: train, val, test')
    label_list = processor.get_labels()

    # if args.task == 'classification':
    labels1 = [float(example.label1) for example in examples]
    labels2 = [int(example.label2) for example in examples]


    batch_encoding = tokenizer.batch_encode_plus(
        [(example.review_token) for example in examples],
        max_length=args.max_seq_len,
        padding="max_length",
        add_special_tokens=True,
        truncation=True,
    )

    features = []
    for i in range(len(examples)):
        inputs = {k: batch_encoding[k][i] for k in batch_encoding}
        
        # For xlm-roberta
        inputs["token_type_ids"] = [0] * len(inputs["input_ids"])  

        feature = InputFeatures(**inputs, label1=labels1[i], label2=labels2[i])
        features.append(feature)

    return features


class Processor(object):
    def __init__(self, args):
        self.args = args

    def get_labels(self):
        if self.args.task == 'classification':
            return CLASSIFICATION_DICT.keys()
        else:
            return [None]

    def _read_file(cls, input_file):
        """Reads a tab separated value file."""
        with open(input_file, "r", encoding="utf-8") as f:
            lines = []
            for line in f:
                lines.append(line.strip())
            return lines

    def _create_examples(self, lines, set_type):
        """Creates examples for the training and dev sets."""
        examples = []
        for (i, line) in enumerate(lines[0:]):
            line = line.split("\t")
            review_token = line[0]
            label1 = line[1]
            label2 = line[2]
            examples.append(InputExample(review_token=review_token, label1=label1
                                         , label2=label2))
        return examples

    def get_examples(self, mode):
        """
        Args:
            mode: train, test
        """
        file_to_read = None
        if mode == "train":
            file_to_read = self.args.train_file
        elif mode == "test":
            file_to_read = self.args.test_file

        return self._create_examples(
            self._read_file(os.path.join(self.args.data_dir, file_to_read)), mode
        )

In [ ]:
tokenizer = args.tokenizer.from_pretrained(
    args.model_name_or_path,
    do_lower_case=args.do_lower_case
)

train_dataset = load_dataset(args, tokenizer, mode="train")
val_dataset = load_dataset(args, tokenizer, mode="test")
datasets = {'train': train_dataset, 'val': val_dataset}

In [ ]:
%load_ext tensorboard

In [ ]:
%tensorboard --logdir=args.tb_root

In [ ]:
import argparse
import json
import logging
import os
import glob
import matplotlib.pyplot as plt
import matplotlib.ticker as mticker
from transformers import (
    AdamW,
    get_linear_schedule_with_warmup
)
from tqdm.auto import tqdm
import numpy as np
import pandas as pd
from torch.utils.data import DataLoader, RandomSampler, SequentialSampler
from fastprogress.fastprogress import master_bar, progress_bar
import torch

# TODO: acc. calculating logic impl. + figure adding logic impl.
        
def train(args, datasets):
    train_sampler = RandomSampler(datasets['train'])
    train_loader = DataLoader(datasets['train'],
                              sampler=train_sampler,
                              batch_size=args.train_batch_size)
    val_sampler = RandomSampler(datasets['val'])
    val_loader = DataLoader(datasets['val'],
                            sampler=val_sampler,
                            batch_size=args.eval_batch_size)
    dataloaders = {'train': train_loader, 'val': val_loader}
    full_text = pd.read_csv(os.path.join(args.data_dir, 'full_text.txt'),
                            sep='\t')
    
    t_total = (
        len(dataloaders['train']) // args.gradient_accumulation_steps
        * args.num_train_epochs
    )

    writer = SummaryWriter(
        os.path.join(args.tb_root, f'version_{args.version}')
        )

    # Load models <model_1: sentiment, model_2: classifiation>
    config_1 = args.config.from_pretrained(args.model_name_or_path,
                                           num_labels=1)
    model_1 = args.model.from_pretrained(args.model_name_or_path,
                                         config=config_1).to(args.device)
    config_2 = args.config.from_pretrained(args.model_name_or_path,
                                           num_labels=16)
    model_2 = args.model.from_pretrained(args.model_name_or_path,
                                         config=config_2).to(args.device)

   
    # Prepare optimizer and schedule (linear warmup and decay)
    no_decay = ['bias', 'LayerNorm.weight']
    optimizer_grouped_parameters_1 = [
        {
            'params': [p for n, p in model_1.named_parameters()
                    if not any(nd in n for nd in no_decay)],
            'weight_decay': args.weight_decay
         },
        {
            'params': [p for n, p in model_1.named_parameters()
                    if any(nd in n for nd in no_decay)],
            'weight_decay': 0.0
         }
    ]
    optimizer_grouped_parameters_2 = [
        {
            'params': [p for n, p in model_2.named_parameters()
                    if not any(nd in n for nd in no_decay)],
            'weight_decay': args.weight_decay
         },
        {
            'params': [p for n, p in model_2.named_parameters()
                    if any(nd in n for nd in no_decay)],
            'weight_decay': 0.0
         }
    ]
    optimizer_1 = AdamW(optimizer_grouped_parameters_1,
                      lr=5e-06,
                      eps=args.adam_epsilon)
    optimizer_2 = AdamW(optimizer_grouped_parameters_2,
                      lr=args.learning_rate,
                      eps=args.adam_epsilon)
    scheduler_1 = get_linear_schedule_with_warmup(
        optimizer_1,
        num_warmup_steps=int(t_total * args.warmup_proportion),
        num_training_steps=t_total
        )
    scheduler_2 = get_linear_schedule_with_warmup(
        optimizer_2,
        num_warmup_steps=int(t_total * args.warmup_proportion),
        num_training_steps=t_total
        )

    if os.path.isfile(os.path.join(args.model_name_or_path, "optimizer.pt")) \
    and os.path.isfile(os.path.join(args.model_name_or_path, "scheduler.pt")):
        # Load optimizer and scheduler states
        optimizer_1.load_state_dict(
            torch.load(os.path.join(args.model_name_or_path, "optimizer.pt"))
            )
        optimizer_2.load_state_dict(
            torch.load(os.path.join(args.model_name_or_path, "optimizer.pt"))
            )
        scheduler_1.load_state_dict(
            torch.load(os.path.join(args.model_name_or_path, "scheduler.pt"))
            )
        scheduler_2.load_state_dict(
            torch.load(os.path.join(args.model_name_or_path, "scheduler.pt"))
            )

    print(f'training started <model_1: sentiment, model_2: classifiation>')
    global_step = 0
    model_1.zero_grad()
    model_2.zero_grad()
    mb = master_bar(range(int(args.num_train_epochs)))
    for epoch in mb:
        mb.main_bar.comment = f'training in progress: epoch {epoch+1}'
        accuracies_1, accuracies_2, losses = {}, {}, {}
        for stage, dataloader in dataloaders.items():
            epoch_iterator = progress_bar(dataloader, parent=mb)
            num_correct_1, num_correct_2, num_items_1, num_items_2 = 0, 0, 0, 0
            running_loss = 0
            for step, batch in enumerate(epoch_iterator):
                mb.child.comment = f'epoch {epoch+1} {stage}'
                if stage == 'train':
                    model_1.train()
                    model_2.train()
                    optimizer_1.zero_grad() # why is this not in original func?
                    optimizer_2.zero_grad()
                else:
                    model_1.eval()
                    model_2.eval()
                
                batch = tuple(t.to(args.device) for t in batch)
                
                inputs_1 = {
                    "input_ids": batch[0],
                    "attention_mask": batch[1],
                    "token_type_ids": batch[2],
                    "labels": batch[3]
                }
                # model output: [loss, grad_fn (MSE), logits, batches]
                outputs_1 = model_1(**inputs_1)
                preds_1 = torch.round(outputs_1.logits).view(-1)
                num_items_1 += preds_1.shape[0]
                num_correct_1 += (inputs_1['labels'] == preds_1).sum()
                loss_1 = outputs_1[0]

                inputs_2 = {
                    "input_ids": batch[0],
                    "attention_mask": batch[1],
                    "token_type_ids": batch[2],
                    "labels": batch[4]
                }
                # model output: [loss, grad_fn (NLL), logits, batches]
                outputs_2 = model_2(**inputs_2)
                preds_2 = torch.argmax(outputs_2.logits, dim=1)
                num_items_2 += preds_1.shape[0]
                num_correct_2 += (inputs_2['labels'] == preds_2).sum()
                loss_2 = outputs_2[0]

                # loss combination + model backward
                loss_1, loss_2 = loss_1 + loss_2, loss_1 + loss_2
                if args.gradient_accumulation_steps > 1:
                    loss = loss / args.gradient_accumulation_steps
                if stage == 'train':
                    loss_1.backward(retain_graph=True)
                    loss_2.backward()
                running_loss += loss_1.item()

                if stage == 'train' and (
                    (step + 1) % args.gradient_accumulation_steps == 0 
                    or (len(dataloader) <= args.gradient_accumulation_steps
                        and (step + 1) == len(dataloader)
                        )
                    ):
                    torch.nn.utils.clip_grad_norm_(model_1.parameters(),
                                                   args.max_grad_norm)
                    torch.nn.utils.clip_grad_norm_(model_2.parameters(),
                                                   args.max_grad_norm)

                    optimizer_1.step()
                    optimizer_2.step()
                    scheduler_1.step()
                    scheduler_2.step()
                    model_1.zero_grad()
                    model_2.zero_grad()
                    global_step += 1
                if args.max_steps > 0 and global_step > args.max_steps:
                    break
            accuracies_1[stage] = num_correct_1 / num_items_1
            accuracies_2[stage] = num_correct_2 / num_items_2
            losses[stage] = running_loss / len(dataloader)
        # Save model checkpoint
        output_dir_1 = os.path.join(args.output_dir,
                                    "1-checkpoint-{}".format(epoch + 1))
        output_dir_2 = os.path.join(args.output_dir,
                                    "2-checkpoint-{}".format(epoch + 1))
        if not os.path.exists(output_dir_1):
            os.makedirs(output_dir_1)
        if not os.path.exists(output_dir_2):
            os.makedirs(output_dir_2)
        model_1_to_save = (
            model_1.module if hasattr(model_1, "module") else model_1
        )
        model_1_to_save.save_pretrained(output_dir_1)
        model_2_to_save = (
            model_2.module if hasattr(model_2, "module") else model_2
        )
        model_2_to_save.save_pretrained(output_dir_2)

        torch.save(args, os.path.join(args.output_dir, "training_args.bin"))
        
        # Tensorboard scalar logging
        writer.add_scalars(main_tag='accuracy_1',
                           tag_scalar_dict=accuracies_1,
                           global_step=epoch+1)
        writer.add_scalars(main_tag='accuracy_2',
                           tag_scalar_dict=accuracies_2,
                           global_step=epoch+1)        
        writer.add_scalars(main_tag='loss',
                          tag_scalar_dict=losses,
                          global_step=epoch+1)
        # print(accuracies_1, accuracies_2)
        if args.max_steps > 0 and global_step > args.max_steps:
            del model_1, model_2
            torch.cuda.empty_cache()
            break

    return

In [ ]:
torch.cuda.empty_cache()
torch.cuda.ipc_collect()
torch.cuda.memory_summary(abbreviated=True)

In [ ]:
# GPU or CPU
args.device = "cuda" if torch.cuda.is_available() and not args.no_cuda else "cpu"


# Train
train(args, datasets)


In [ ]:
def test(args, model, test_data, mode, global_step=None):
  results = {}
  checkpoints = list(
              os.path.dirname(c) for c in
              sorted(glob.glob(args.output_dir + "/**/" + "pytorch_model.bin", recursive=True))
          )
  print(checkpoints)
  checkpoint = checkpoints[-1]
  logger.info("Test the following checkpoint: %s", checkpoint)
  global_step = checkpoint.split("-")[-1]
  model = args.model.from_pretrained(checkpoint)
  model.to(args.device)

  test_sampler = SequentialSampler(test_data)
  test_dataloader = DataLoader(test_data, sampler=test_sampler, batch_size=args.eval_batch_size)

  # Eval!
  if global_step != None:
      logger.info("***** Running Test on {} dataset ({} step) *****".format(mode, global_step))
  else:
      logger.info("***** Running test on {} dataset *****".format(mode))
  logger.info("  Num examples = {}".format(len(test_data)))
  logger.info("  Eval Batch size = {}".format(args.eval_batch_size))
  eval_loss = 0.0
  # nb_eval_steps = 0
  preds = None
  preds_save = None
  out_input_ids = None

  for batch in progress_bar(test_dataloader):
      preds_temp = None
      model.eval()
      batch = tuple(t.to(args.device) for t in batch)

      with torch.no_grad():
          inputs = {
              "input_ids": batch[0],
              "attention_mask": batch[1],

              # label 없이 할 땐 여기 주석 반전
              "labels": batch[3]

          }

          # if args.model_type not in ["distilkobert", "xlm-roberta"]:
          inputs["token_type_ids"] = batch[2]  # Distilkobert, XLM-Roberta don't use segment_ids

          outputs = model(**inputs)

          # label 없이 할 땐 여기 주석 반전
          tmp_eval_loss, logits = outputs[:2]
          # logits = outputs[0]
          eval_loss += tmp_eval_loss.mean().item()

      # nb_eval_steps += 1
      if preds is None:
          out_input_ids = inputs['input_ids'].detach().cpu().numpy()
          preds = logits.detach().cpu().numpy()
          preds_save = preds.copy()

          # label 없이 할 땐 여기 주석 반전
          out_label_ids = inputs["labels"].detach().cpu().numpy()

      else:
          out_input_ids = np.append(out_input_ids, inputs['input_ids'].detach().cpu().numpy(), axis=0)
          preds_temp = logits.detach().cpu().numpy()
          preds = np.append(preds, logits.detach().cpu().numpy(), axis=0)
          preds_save = np.append(preds_save, logits.detach().cpu().numpy(), axis=0)

          # label 없이 할 땐 여기 주석 반전
          out_label_ids = np.append(out_label_ids, inputs["labels"].detach().cpu().numpy(), axis=0)

  if args.task == 'sentiment':
    for i in range(len(preds)):
      if preds[i][0] < 0.5:
        preds[i][0] = 0
      else:
        preds[i][0] = 1
  preds_original = preds.copy()
  if args.task == 'classification':
    preds = np.argmax(preds, axis=1)
  else:
    preds = np.squeeze(preds)
  
  # label 없이 할 땐 return 전 까지 주석 반전
  result = compute_metrics(out_label_ids, preds)
  tokenizer = args.tokenizer.from_pretrained(
      args.model_name_or_path,
      do_lower_case=args.do_lower_case
  )
  # label_dict = {'칭찬': 0, '중립': 0.5, '불만': 1}
  if args.task == 'classification':
    label_dict = {'중립': 0, '상담원': 1, '상담시스템': 2, '혜택': 3, '할부금융상품': 4,
                '카드상품': 5, '청구입금': 6, '심사/한도': 7, '생활편의서비스': 8,
                '상담/채널': 9, '리스렌탈상품': 10, '라이프서비스': 11, '금융상품': 12,
                '고객정보관리': 13, '가맹점매출/승인': 14, '삼성카드': 15, '기타': 16}
  else:
    label_dict = {'칭찬': 0, '불만': 1}
  label_dict = dict((v, k) for k, v in label_dict.items())
  check_right = out_label_ids == preds
  df = pd.DataFrame(columns=['Review', 'Label', 'Prediction', 'Softmax'])
  for check_result in range(len(check_right)):
      if not check_right[check_result]:
          review_list = list(out_input_ids[check_result])
          while 0 in review_list:
              review_list.remove(0)
          del review_list[0]
          del review_list[-1]
          df = df.append({'Review': tokenizer.decode(review_list), 
                          'Label': label_dict[out_label_ids[check_result]], 
                          'Prediction': label_dict[preds[check_result]],
                          'Softmax': preds_save[check_result]},
                          ignore_index=True
                          )
  
  print(df.head())

  result = dict((k + "_{}".format(global_step), v) for k, v in result.items())
  results.update(result)
  print('test accuracy :', result)
  return preds

In [ ]:
test_result = test(args, model, val_dataset, 'val')

In [ ]:
list_preds = []

In [ ]:
list_preds.append(test_result)


In [ ]:
print(len(list_preds[0]))

In [ ]:
def complete_result(list_preds):
  label_dict_classification = {'중립': 0, '상담원': 1, '상담시스템': 2, '혜택': 3, '할부금융상품': 4,
                '카드상품': 5, '청구입금': 6, '심사/한도': 7, '생활편의서비스': 8,
                '상담/채널': 9, '리스렌탈상품': 10, '라이프서비스': 11, '금융상품': 12,
                '고객정보관리': 13, '가맹점매출/승인': 14, '삼성카드': 15, '기타': 16}
  label_dict_sentiment = {'칭찬': 0, '불만': 1}
  label_dict_classification = dict((v, k) for k, v in label_dict_classification.items())
  label_dict_sentiment = dict((v, k) for k, v in label_dict_sentiment.items())
  for i in label_dict_classification:
    if i not in [0, 1, 2, 16]:
      label_dict_classification[i] = '>삼성카드>' + label_dict_classification[i]
    else:
      if i == (1 or 2):
        label_dict_classification[i] = '>고객서비스>' + label_dict_classification[i]
      if i == 16:
        label_dict_classification[i] = '>' + label_dict_classification[i]
  df = pd.DataFrame(columns = ['INT'])
  sent_result = np.vectorize(label_dict_sentiment.get)(list_preds[0])
  # class_result = np.vectorize(label_dict_classification.get)(list_preds[1])
  # result_arr = np.core.defchararray.add(sent_result, class_result)
  df['INT'] = sent_result
  print(df)

In [ ]:
complete_result(list_preds)

In [ ]:
# 수정금지: 타임스탬프
_model_build_end_time = printt("Model building: Start")
print(_model_build_end_time - _model_build_start_time)

#모델 분류

In [ ]:
# 수정금지: 타임스탬프
_test_start_time = printt("TEST: Start")

In [ ]:
#TODO: 해당 블럭에 테스트 수행을 위한 코드를 넣으세요. (시간측정 구간)
#분석 파일은 tsv 파일로 제공되며, 제공되는 학습데이터 파일과 동일한 레이아웃(단, 정답 분류 =공백)으로 제공됩니다.




In [ ]:
# 수정금지: 타임스탬프
_test_end_time = printt("Model building: Start")
print(_test_end_time - _test_start_time)

# 결과출력

In [ ]:
#TODO:해당 블럭에 테스트 결과를 파일로 저장하는 코드를 넣으세요. (시간측정 제외)
#저장 파일은tsv 파일로 제공되는 학습데이터 파일과 동일한 레이아웃(단, 정답 분류 = 테스트 결과 도출된 양식)으로 저장


